Single

In [ ]:
# Workflow for DRR with Bone Adjustment

# 1. Data Extraction
ct_no_bed_folder = r"D:\drr_real_study\single_CT"
sep_ct_folder = r"D:\drr_real_study\Sep_cts"
patient_name = "101_a_1"

full_ct, bones_array_cp, spine_array_cp, rib_array_cp, soft_tissue_array_cp, affine, header = extract_patient_data(
    ct_no_bed_folder, sep_ct_folder, patient_name
)

# 2. Surface Enhancement
bones_array_new = enhance_surface(bones_array_cp, adjustment_factor=1.1, inner_percentage=3)
rib_array_new = enhance_surface(rib_array_cp, adjustment_factor=1.1, inner_percentage=3)
spine_array_new = enhance_surface(spine_array_cp, adjustment_factor=1.1, inner_percentage=3)

# 3. Vertical Transition Adjustment (Spine)
spine_array_new = adjust_vertical_transition(
    spine_array_new, vertical_point_lower=0.7, adjustment_factor=0.2, top_down=1, speed=1.5
)

# 4. Image Combination
combined_img = combine_img(bones_array_new, spine_array_new, rib_array_new, soft_tissue_array_cp, affine)

# 5. DRR Generation and Plotting
img = generate_drr(
    combined_img,
    header,
    orientation,
    bone_multi,
    [[3.0, 0.0, 0.0]],
    translate=[[0.0, 1220 - 167.2, 0.0]],
    device=device,
    sdd=1220
)
plot_drr_large(img, figsize=(10, 10), title=None, ticks=False)


In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np

def process_patient(
    full_ct, bones_array_cp, spine_array_cp, rib_array_cp, soft_tissue_array_cp, affine, header,
    inner_percentages, adjustment_factors, save_dir, orientation, bone_multi, device, sdd=1220
):
    """
    Process a single patient's data with multiple inner_percentage and adjustment_factor values.

    Parameters:
        full_ct, bones_array_cp, spine_array_cp, rib_array_cp, soft_tissue_array_cp, affine, header:
            Data extracted using `extract_patient_data`.
        inner_percentages (list): List of inner_percentage values to compute inner surfaces.
        adjustment_factors (list): List of adjustment_factor values for intensity adjustment.
        save_dir (str): Directory to save the resulting plots.
        orientation: DRR orientation parameter.
        bone_multi: DRR bone multiplier parameter.
        device: GPU device to use for DRR generation.
        sdd (float): Source-to-detector distance for DRR.
    """
    # Ensure save directory exists
    os.makedirs(save_dir, exist_ok=True)

    # Step 1: Precompute the surface mask (common for all iterations)
    print("Precomputing surface mask...")
    surface_mask, _, _ = find_surface(bones_array_cp)  # Example for bones

    # Divide adjustment factors into subsets for each inner_percentage
    adjustment_factor_groups = np.array_split(adjustment_factors, len(inner_percentages))


    # Step 2: Loop through inner_percentages and their corresponding adjustment factors
    for i, inner_percentage in enumerate(inner_percentages):
        print(f"Processing inner_percentage={inner_percentage}...")

        # Compute inner surface for this percentage
        binary_mask = bones_array_cp > 0
        inner_surface = find_inner_surface(binary_mask, inner_percentage=inner_percentage)

        # Get the corresponding adjustment factors
        current_factors = adjustment_factor_groups[i]

        # Process for each adjustment factor
        for adjustment_factor in current_factors:
            print(f"  Applying adjustment_factor={adjustment_factor}...")

            # Adjust bone, rib, and spine surfaces
            bones_array_new = adjust_surface_intensity(
                bones_array_cp, surface_mask, inner_surface, adjustment_factor
            )
            rib_array_new = adjust_surface_intensity(
                rib_array_cp, surface_mask, inner_surface, adjustment_factor
            )
            spine_array_new = adjust_surface_intensity(
                spine_array_cp, surface_mask, inner_surface, adjustment_factor
            )

            # Combine arrays
            combined_img = combine_img(bones_array_new, spine_array_new, rib_array_new, soft_tissue_array_cp, affine)

            # Generate DRR
            img = generate_drr(
                combined_img, header, orientation, bone_multi, [[3.0, 0.0, 0.0]],
                translate=[[0.0, sdd - 167.2, 0.0]], device=device, sdd=sdd
            )

            # Plot and save DRR
            output_file = os.path.join(save_dir, f"DRR_{inner_percentage}_{adjustment_factor}.png")
            plot_drr_large(img, figsize=(10, 10), title=None, ticks=False)
            plt.savefig(output_file)
            plt.close()

            # Explicitly clear variables for memory efficiency
            del bones_array_new, rib_array_new, spine_array_new, combined_img, img

        # Clear inner_surface after processing the current inner_percentage
        del inner_surface
        print(f"Completed inner_percentage={inner_percentage}.")

    print("Processing complete for this patient.")

# Example usage:
# Assuming `extract_patient_data` is called and its results are passed:
# inner_percentages = [1, 2, 3, 4, 5]
# adjustment_factors = [1.0, 1.02, ..., 2.0]  # 100 values
# process_patient(
#     full_ct, bones_array_cp, spine_array_cp, rib_array_cp, soft_tissue_array_cp, affine, header,
#     inner_percentages, adjustment_factors, save_dir="output_dir", orientation="AP", 
#     bone_multi=2.0, device="cuda:0", sdd=1220
# )
